In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
import os
import json
import math

cwd = os.getcwd()
business_path = cwd + "/a11_cleaned_businesses.json"
reviews_path = cwd + "/a1_filtered_reviewsv3.json"
users_path = cwd + "/a1_cleaned_users.json"

businesses = pd.read_json(business_path, lines=True)
reviews = pd.read_json(reviews_path, lines=True)
users = pd.read_json(users_path, lines=True)

## Se sigue la fórmula especificada en el paper

In [3]:
count_businesses = len(businesses)
businesses["categories"]

0         Shipping Centers, Local Services, Notaries, Ma...
1         Department Stores, Shopping, Fashion, Home & G...
2         Restaurants, Food, Bubble Tea, Coffee & Tea, B...
3                                 Brewpubs, Breweries, Food
4         Burgers, Fast Food, Sandwiches, Food, Ice Crea...
                                ...                        
147856                           Nail Salons, Beauty & Spas
147857    Pets, Nurseries & Gardening, Pet Stores, Hobby...
147858    Shopping, Jewelry, Piercing, Toy Stores, Beaut...
147859    Fitness/Exercise Equipment, Eyewear & Optician...
147860    Beauty & Spas, Permanent Makeup, Piercing, Tattoo
Name: categories, Length: 147861, dtype: object

In [4]:
MIN_BUSINESSES_PER_CATEGORY = 15 # Para limpieza y reduccion de dimensionalidad

businesses["categories"] = businesses["categories"].str.split(", ")

category_counts = defaultdict(int)
for cats in businesses["categories"]:
    if isinstance(cats, list):
        for cat in cats:
            category_counts[cat] += 1


valid_categories = {cat for cat, count in category_counts.items() 
                    if count >= MIN_BUSINESSES_PER_CATEGORY}

print(f"Categorias finales {len(valid_categories)}")

Categorias finales 983


In [5]:
category_count =defaultdict(int)

for place in businesses.iterrows():
    if place[1]['categories']:
        for category in place[1]['categories']:
            category_count[category] += 1

In [6]:
idf = {category: math.log(len(businesses) / count) for category, count in category_count.items()}
# idf

In [10]:
def compute_user_confidence(users):
    max_friends = users['friends'].apply(len).max()
    
    confidence_scores = []
    for _, user in users.iterrows():
        review_count = user['review_count']
        friend_count = len(user['friends'])
        
        activity = math.log(1 + review_count)
        social_influence = 1 + (friend_count / max_friends)
        confidence_scores.append(activity * social_influence)
    
    users['confidence_score'] = confidence_scores
    return users
    
users_wconfidence = compute_user_confidence(users).drop(['compliment_hot'], axis=1) # No sé por qué esa columna no estaba eliminada

In [11]:
 print("Mínimo user_confidence->",min(users_wconfidence['confidence_score']),"\nMáximo user_confidence->",max(users_wconfidence['confidence_score']))

Mínimo user_confidence-> 0.6931548848061683 
Máximo user_confidence-> 14.243344186805276


In [13]:
#Esto solo lo hice por curiosidad. Me impresionó que hubieran tantos usuarios con una cantidad absurda de amigos agregada.
def count_users_with_friends(df, min_friends_count):
    counter = 0
    for index, row in df.iterrows():
        if len(row['friends']) >= min_friends_count:
            counter += 1
    return counter

min_friends = 10000
result = count_users_with_friends(users, min_friends)

print(f"Hay {result} usuarios con al menos {min_friends} amigos")

Hay 9582 usuarios con al menos 10000


In [16]:
def compute_global_avg_rating(reviews):
    total_rating = sum(review['stars'] for index,review in reviews.iterrows())
    total_reviews = len(reviews)
    return total_rating / total_reviews if total_reviews > 0 else 0

global_avg = compute_global_avg_rating(reviews)
global_avg

3.8283277576628456

In [17]:
type(businesses["categories"][0])

list

In [19]:
## ---- ❤ Deepseek: ❤  ---- 

def build_business_profiles(businesses_df, reviews_df, users_df, idf_dict, global_avg_rating, alpha=5):
    user_lookup = pd.Series(users_df.confidence_score.values, index=users_df.user_id).to_dict()
    
    reviews_grouped = reviews_df.groupby('business_id')
    
    business_profiles = []
    
    for _, biz_row in businesses_df.iterrows():
        profile = {}
        business_id = biz_row['business_id']
        
        try:
            biz_reviews = reviews_grouped.get_group(business_id)
        except KeyError:
            biz_reviews = pd.DataFrame()

        # Bayesian-adjusted rating calculation
        weighted_sum = 0
        confidence_sum = 0
        
        for _, review_row in biz_reviews.iterrows():
            user_id = review_row['user_id']
            if user_id in user_lookup:
                weighted_sum += review_row['stars'] * user_lookup[user_id]
                confidence_sum += user_lookup[user_id]

        # Handle division by zero and new businesses
        denominator = confidence_sum + alpha
        adj_rating = (weighted_sum + alpha * global_avg_rating) / denominator if denominator != 0 else global_avg_rating

        # Process categories (lists or NaN)
        categories = []
        if isinstance(biz_row['categories'], list):
            # Filter out empty strings and whitespace-only categories
            categories = [c.strip() for c in biz_row['categories'] if c.strip()]
        elif pd.isna(biz_row['categories']):
            # Handle NaN values
            categories = []
        
        # Calculate category weights
        for category in categories:
            if category in idf_dict:
                tfidf = idf_dict[category]
                popularity = math.log(1 + len(biz_reviews))
                profile[category] = float(tfidf * adj_rating * popularity)

        # Add to profiles
        business_profiles.append({
            'business_id': business_id,
            'profile': profile
        })
    
    # Write to JSON file
    with open('business_profiles.json', 'w') as f:
        json.dump(business_profiles, f, indent=2, default=float)
    
    return business_profiles

build_business_profiles(businesses,reviews,users,idf,global_avg)

[{'business_id': 'mpf3x-BjTdTEA3yCZrAYPw',
  'profile': {'Shipping Centers': 34.55023657088548,
   'Local Services': 17.200290865552574,
   'Notaries': 37.34006581041501,
   'Mailbox Centers': 40.10802871775396,
   'Printing Services': 34.592644921996964}},
 {'business_id': 'tUFrWirKiKi_TAnsVWINQQ',
  'profile': {'Department Stores': 50.50313872763138,
   'Shopping': 19.785695690150067,
   'Fashion': 35.46433820823066,
   'Home & Garden': 35.43183265675176,
   'Electronics': 50.71102039490446,
   'Furniture Stores': 49.4707646605216}},
 {'business_id': 'MTSW4McQd7CbVtyjqoe9mw',
  'profile': {'Restaurants': 17.659845030853685,
   'Food': 28.392792955109563,
   'Bubble Tea': 97.30712801464747,
   'Coffee & Tea': 52.50081072481885,
   'Bakeries': 65.32523947678864}},
 {'business_id': 'mWMc6_wTdE0EUBKIGXDVfA',
  'profile': {'Brewpubs': 55.04266379773139,
   'Breweries': 42.37878204145722,
   'Food': 13.784646473846854}},
 {'business_id': 'CF33F8-E6oudUQ46HnavjQ',
  'profile': {'Burgers': 1

In [26]:
# De forma similar a la normalizacion de los usuarios
def normalize_business_profiles(input_file, output_file):
    with open(input_file, 'r') as f:
        businesses = json.load(f)
    
    valid_businesses = []
    empty_count = 0
    
    for biz in businesses:
        profile = biz['profile']
        
        if not profile:
            empty_count += 1
            continue
            
        # Norma L2
        squared_sum = sum(value ** 2 for value in profile.values())
        l2_norm = math.sqrt(squared_sum)
        
        if l2_norm > 0:
            normalized_profile = {
                category: value / l2_norm
                for category, value in profile.items()
            }
            valid_businesses.append({
                'business_id': biz['business_id'],
                'profile': normalized_profile
            })
        else:
            print(f"Warning: Zero norm for business {biz['business_id']} - profile discarded")
            empty_count += 1
    
    with open(output_file, 'w') as f:
        json.dump(valid_businesses, f, indent=2)
    
    print(f"Processed {len(businesses)} businesses")
    print(f"Removed {empty_count} empty/invalid profiles")
    print(f"Saved {len(valid_businesses)} normalized profiles to {output_file}")


business_profiles_path = cwd + "/a11_cleaned_businesses.json"

normalize_business_profiles("business_profiles.json","business_profiles_l2_normalized.json")

Processed 147861 businesses
Removed 87 empty/invalid profiles
Saved 147774 normalized profiles to business_profiles_l2_normalized.json
